In [157]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")
import math


In [56]:
# 讀取SizeData
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\20230710_SizeData.xlsx", None)

In [57]:
df = df["Sheet1"]

In [65]:
df = df[df["通路明細"] != "LSCM"]

In [66]:
df["花紋"] = df["花紋"].map(lambda x: x[:4])

In [67]:
df["position_up"] = df["花紋"].map(lambda x: "New" if x == "R154" or x == "UAP1" else "No")

In [68]:
ch_key = ['FLT-REP', 'AG', 'A', 'FMC', '一般店', 'BSSC-FLT', 'CD', '中盤', 'DS',
       'TK', 'MM', 'FMC BFP', 'FLT', 'TTC', 'BFP', '量販店', 'OT',
       'FLT-BSSC', '加油站', '輪胎行', 'FT']
ch_value = ['FLT-REP', 'AG', 'BSSC', 'BSSC', 'BSSC', 'BSSC-FLT', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC-FLT', 'BSSC', 'BSSC', 'BSSC', 
            'FLT-REP', 'BSSC-FLT', 'BSSC', 'BSSC', 'FLT-REP']
ch_dict = dict(zip(ch_key, ch_value))

In [70]:
df["大通路"] = df["通路明細"].map(ch_dict)

In [72]:
df = df[df["預算/前實績"] == "23ACT"]

In [73]:
df["舊分類"] = df["舊分類"].map(lambda x: "LSR" if x == "LSR進口品" else x)

In [74]:
df = df[["年月", "舊分類", "商品代號", "大通路", "position_up", "NET金額", "條數"]]

In [75]:
df = df.fillna(0)

In [17]:
# 讀取RBA&C 2
rt2 = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\RT2\20230320_RT2_assumption v2.xlsx", None)["RBA simulator with OP"]


In [18]:
rt2["position_up"] = rt2["花紋"].map(lambda x: "New" if x == "R154" or x == "UAP1" else "No")

In [19]:
rt2 = rt2[["新年月", "Type", "PCODE", "大通路", "position_up", "NET Total", "數量"]]

In [20]:
rt2.columns = df.columns

In [91]:
# 篩選基準和對比資料
# budget_df = df[(df["年月"] == 202305)]
actual_df = df[(df["年月"] == 202306)]

budget_df = rt2[(rt2["年月"] == 202306)]
# actual_df = rt2[(rt2["年月"]== 202307)]

In [92]:
# 複製貼上有要做販促的PCODE
promotion = pd.read_clipboard()["PCODE"].values.tolist()
budget_df["promotion"] = budget_df["商品代號"].map(lambda x: "Yes" if x in promotion else "No")

In [93]:
promotion = pd.read_clipboard()["PCODE"].values.tolist()
actual_df["promotion"] = actual_df["商品代號"].map(lambda x: "Yes" if x in promotion else "No")

In [94]:
final = pd.concat([budget_df, actual_df])

In [95]:
budget_asp = round(budget_df["NET金額"].sum() / budget_df["條數"].sum(), 1)
actual_asp = round(actual_df["NET金額"].sum() / actual_df["條數"].sum(), 1)

In [96]:
print(budget_asp, actual_asp)
print(round((actual_asp / budget_asp), 1))

9735.2 8755.0
0.9


In [97]:
# 預算和實績ASP差異
diff = actual_asp - budget_asp


In [98]:
diff

-980.2000000000007

In [149]:
# 創建一個空字典來存儲貢獻值
contribution = {}

In [150]:
budget_df

,年月,舊分類,商品代號,大通路,position_up,NET金額,條數,promotion
357,202306,LSR,LSR01180,AG,No,119069,22,Yes
358,202306,LSR,LSR01180,BSSC-FLT,No,23409,4,Yes
359,202306,LSR,LSR01180,BSSC,No,1199232,216,Yes
360,202306,LSR,LSR01180,BSSC,No,433056,78,Yes
361,202306,LSR,LSR01270,AG,No,59039,10,Yes
...,...,...,...,...,...,...,...,...
472,202306,TBR,TBR0J660,FLT-REP,No,42318,4,No
473,202306,TBR,TBR0J660,BSSC,No,454897,36,No
474,202306,TBR,TBR0J781,BSSC-FLT,New,340209,30,No
475,202306,TBR,TBR0J781,BSSC,New,4230015,368,No


In [151]:
budget_df[budget_df["大通路"] == "AG"]

,年月,舊分類,商品代號,大通路,position_up,NET金額,條數,promotion
357,202306,LSR,LSR01180,AG,No,119069,22,Yes
361,202306,LSR,LSR01270,AG,No,59039,10,Yes
365,202306,LSR,LSR03459,AG,No,162375,23,Yes
372,202306,LSR,LSR03765,AG,No,384228,60,Yes
377,202306,LSR,LSR05127,AG,No,59926,10,No
...,...,...,...,...,...,...,...,...
441,202306,TBR,TBR0J295,AG,No,95629,10,No
446,202306,TBR,TBR0J314,AG,No,91560,8,Yes
454,202306,TBR,TBR0J418,AG,No,101811,8,No
456,202306,TBR,TBR0J487,AG,No,276250,24,Yes


In [152]:
actual_df[actual_df["大通路"] == "AG"]["條數"].sum()

0

In [158]:
# 計算客戶舊分類、大通路、position up和promotion對差異的貢獻值
# 將需要計算的欄位寫在list裡面然後建立一個loop
# 將list裡面每個欄位裡面的unique value計算基準和對比的asp
# 將對比和基準的asp做相減得到差異
# 如果差異大於0，儲存在positive的keys，小於0的存在negative的keys
for column in ['舊分類', '大通路', 'position_up', 'promotion']:
    for category in final[column].unique():
        category_budget = budget_df[budget_df[column] == category]
        category_budget_asp = category_budget["NET金額"].sum() / category_budget["條數"].sum()
        # print(category_budget_asp)
        category_actual = actual_df[actual_df[column] == category]
        category_actual_asp = category_actual["NET金額"].sum() / category_actual["條數"].sum()
        # print(category_actual_asp)
        if pd.isna(category_actual_asp) and category_budget_asp != 0:
            category_difference = category_budget_asp * -1
        elif pd.isna(category_budget_asp) and category_actual_asp != 0:
            category_difference = category_actual_asp
        elif math.isinf(category_actual_asp):
            category_difference = 0
        else:
            category_difference = category_actual_asp - category_budget_asp
        # print(category, category_difference)
        contribution[f"{column}+{category}"] = category_difference if category_difference > 0 else 0
        contribution[f"{column}-{category}"] = category_difference if category_difference < 0 else 0

In [159]:
contribution

{'舊分類+LSR': 0,
 '舊分類-LSR': -406.3265526932082,
 '舊分類+TBR': 0,
 '舊分類-TBR': -1244.0734840798305,
 '大通路+AG': 0,
 '大通路-AG': -8349.017064846417,
 '大通路+BSSC-FLT': 0,
 '大通路-BSSC-FLT': -890.071499623542,
 '大通路+BSSC': 0,
 '大通路-BSSC': -1494.848689705972,
 '大通路+FLT-REP': 0,
 '大通路-FLT-REP': -138.37615033543625,
 'position_up+No': 0,
 'position_up-No': -856.7931260817495,
 'position_up+New': 0,
 'position_up-New': 0,
 'promotion+Yes': 0,
 'promotion-Yes': -385.64476741924227,
 'promotion+No': 0,
 'promotion-No': -2556.773172905526}

In [160]:
sum(contribution.values())

-16321.924507690925

In [161]:
# 總和貢獻值，計算所有keys的value，相加起來應該會等於基準和對比的asp的差異總和
contribution_sum = sum(contribution.values())

# 將貢獻值標準化為百分比，遍歷字典的每個key，然後計算每個value的百分比再乘上基準和對比的差異
contribution_percentages = {key: (value / contribution_sum) * diff for key, value in contribution.items()}

# 輸出結果
print(f"預算平均銷售NET價格：{budget_asp:.2f}")
print(f"實績平均銷售NET價格：{actual_asp:.2f}")
print(f"差異金額：{diff:.2f}")
for key, value in contribution_percentages.items():
    print(f"{key}的貢獻值：{value:.2f}")
# print(f"貢獻值總和:{sum(contribution_percentages.values()):.2f}")

預算平均銷售NET價格：9735.20
實績平均銷售NET價格：8755.00
差異金額：-980.20
舊分類+LSR的貢獻值：0.00
舊分類-LSR的貢獻值：-24.40
舊分類+TBR的貢獻值：0.00
舊分類-TBR的貢獻值：-74.71
大通路+AG的貢獻值：0.00
大通路-AG的貢獻值：-501.39
大通路+BSSC-FLT的貢獻值：0.00
大通路-BSSC-FLT的貢獻值：-53.45
大通路+BSSC的貢獻值：0.00
大通路-BSSC的貢獻值：-89.77
大通路+FLT-REP的貢獻值：0.00
大通路-FLT-REP的貢獻值：-8.31
position_up+No的貢獻值：0.00
position_up-No的貢獻值：-51.45
position_up+New的貢獻值：0.00
position_up-New的貢獻值：0.00
promotion+Yes的貢獻值：0.00
promotion-Yes的貢獻值：-23.16
promotion+No的貢獻值：0.00
promotion-No的貢獻值：-153.54


In [162]:
pro_mix = 0
ch_mix = 0
pos_up = 0
discount = 0
for key, value in contribution_percentages.items():
    if '舊分類' in key:
        pro_mix += value
    elif '大通路' in key:
        ch_mix += value
    elif 'position_up' in key:
        pos_up += value
    else:
        discount += value
        
    

In [163]:
[pro_mix, ch_mix, pos_up, discount]

[-99.1134418789071, -652.928004542628, -51.454019517710826, -176.7045340607548]

In [164]:
print('product mix is {}'.format(round(pro_mix, 0)))
print('channel mix is {}'.format(round(ch_mix, 0)))
print('position up is {}'.format(round(pos_up, 0)))
print('discount is {}'.format(round(discount, 0)))

product mix is -99.0
channel mix is -653.0
position up is -51.0
discount is -177.0


In [148]:
pd.DataFrame([pos_up, 0, ch_mix, pro_mix, discount]).to_clipboard(index=False)